In [1]:
!pip3 install pymysql
import pymysql as MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.8f}'.format

    100% |████████████████████████████████| 51kB 12.6MB/s ta 0:00:01
You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
def SQL_TO_DF (SQL, cnx):
    
    cursor = cnx.cursor() 
    cursor.execute(SQL)
    values = list(cursor.fetchall())
    header = cursor.description
    cursor.close()
    columns = [i[0] for i in header]
    raw = pd.DataFrame(values, columns = columns)
    return raw

In [3]:
cnx = MySQLdb.connect(user='admin', password='rdx-mysql',
                              host='database-1.ccbb7nilplok.us-east-1.rds.amazonaws.com',
                              database='etl')

In [4]:
query = ("select * from event_incident_mapping_week2 where METRIC_NAME IN ('PctFree','dskPercent','used','utilization','PctTotal','LDldFreeSpacePct','UsedMBPct','LDldDiskTimePercent')")
raw = SQL_TO_DF(query, cnx)
query = ("select * from incidents_week2")
incidents = SQL_TO_DF(query, cnx)

In [5]:
# raw.columns

In [6]:
mapped_events = raw[raw['HAS_MAPPED_INCIDENT'] == True][['POLLER_TIME','RECON_ID','METRIC_NAME']].drop_duplicates()

In [7]:
raw = raw.reset_index()

In [8]:
merge = raw.merge(mapped_events, on = ['RECON_ID','METRIC_NAME'], how = 'left')
merge['DIFF'] = merge['POLLER_TIME_x'] - merge['POLLER_TIME_y']
merge['HAS_NEIGHBOR_INCIDENT'] = (merge['DIFF'] < 3600) & (merge['DIFF'] > -3600)

In [9]:
trimmed = merge[['index','POLLER_TIME_x','METRIC_NAME','MEASURED_VALUE','RECON_ID','NEW_BUS_ID','SITE','HAS_NEIGHBOR_INCIDENT','HAS_MAPPED_INCIDENT']]
trimmed = trimmed.drop_duplicates()
trimmed['MEASURED_VALUE'] = trimmed['MEASURED_VALUE'].astype(float)
trimmed = trimmed[trimmed['MEASURED_VALUE'] <= 105]

In [10]:
trimmed = pd.get_dummies(trimmed, columns=['METRIC_NAME']).join(trimmed['METRIC_NAME'])
for METRIC_NAME in ['LDldDiskTimePercent', 'dskPercent', 'utilization', 'PctTotal','PctFree', 'used', 'LDldFreeSpacePct', 'UsedMBPct']:
        trimmed[METRIC_NAME+'_VALUE'] = trimmed['METRIC_NAME_'+METRIC_NAME] * trimmed['MEASURED_VALUE'] 
        trimmed[METRIC_NAME+'_VALUE'].replace(0, np.nan, inplace=True)

In [11]:
trimmed = trimmed[['POLLER_TIME_x','MEASURED_VALUE',
       'RECON_ID', 'NEW_BUS_ID', 'SITE','HAS_NEIGHBOR_INCIDENT',
       'LDldDiskTimePercent_VALUE', 'dskPercent_VALUE',
       'utilization_VALUE', 'PctTotal_VALUE',
       'PctFree_VALUE', 'used_VALUE',
       'LDldFreeSpacePct_VALUE', 'UsedMBPct_VALUE','METRIC_NAME']]

In [12]:
trimmed['HOUR'] =pd.to_datetime(trimmed['POLLER_TIME_x'], unit='s').dt.hour 
trimmed['WEEKDAY'] =pd.to_datetime(trimmed['POLLER_TIME_x'], unit='s').dt.weekday 

In [13]:
final_dataset = trimmed.drop(['RECON_ID','METRIC_NAME'], axis = 1)
final_dataset.dropna(how='all', axis=1)
final_dataset.loc[final_dataset['SITE'].isin(['MMSB400MinutemanRd','MM400MinutemanRoad','MMSB400MinutemanRd']),'SITE'] = 'MM400MinutemanRd'
final_dataset.loc[final_dataset['SITE'].isin(['SCCA1302805LafayetteSt','SCCA1202805LafayetteSt','SCCA1102805LafayetteSt']),'SITE'] = 'SCCALafayetteSt'

In [14]:
final_dataset.rename(columns = {"POLLER_TIME_x":"POLLER_TIME"}, inplace = True)

In [15]:
final_dataset.to_csv('./final_dataset/week2.csv')